## Este código limpia y acota las bases de datos del código de sector 72 del NAICS

La diferencia con el archivo de limpieza_turismo es el periodo de tiempo de los datos que estoy ocupando. Asimismo, la base final de este código es una base distinta de la cual se está obteniendo los resultados principales. Esta base tiene el objetivo de explicar por qué Airbnb pudo haber reducido el crimen.

In [8]:
import pandas as pd
import os

In [9]:
#Definimos la carpeta en donde están los datos
carpeta = r"C:\Users\santo\Documents\CIDE\Tesis\datos_turismo"

In [10]:
#Listar las columnas que necesito
columnas_deseadas = ['id', 'latitud', 'longitud', 'cve_ent', 'codigo_act']

#Listar los archivos que necesito
archivos = sorted([f for f in os.listdir(carpeta) if f.startswith("denue_inegi_72") and f.endswith(".csv") and "_2015" not in f])

dataframe = []

for archivo in archivos:
    ruta = os.path.join(carpeta, archivo)
    
    # Extraer año y mes desde el nombre (e.g. 'denue_inegi_72_2019_11.csv')
    partes = archivo.replace(".csv", "").split("_")
    year, mes = partes[3], partes[4]

    if mes == "04":
        mes = "03"
    elif mes == "11":
        mes = "09"
    
    fecha = pd.to_datetime(f"{year}-{mes}").to_period("M")        
    df = (pd.read_csv(ruta, usecols=columnas_deseadas, encoding="latin-1")
          .query("cve_ent == 9")
          .query("codigo_act >= 722000 and codigo_act < 723000") ######### <------ es esta
          .assign(mes=fecha)
    )
    dataframe.append(df)

df_concatenado = pd.concat(dataframe, ignore_index=True)
df_concatenado

,id,codigo_act,cve_ent,latitud,longitud,mes
0,8332611,722513,9,19.452173,-99.218700,2019-03
1,8332182,722513,9,19.510482,-99.137581,2019-03
2,8332181,722513,9,19.466036,-99.186572,2019-03
3,8536060,722513,9,19.403459,-99.155364,2019-03
4,6707848,722515,9,19.421814,-99.160515,2019-03
...,...,...,...,...,...,...
174814,8584276,722511,9,19.307901,-99.163911,2020-03
174815,7743173,722515,9,19.449317,-99.162096,2020-03
174816,736223,722519,9,19.328171,-99.043536,2020-03
174817,8368771,722515,9,19.396080,-99.171465,2020-03


## Preparamos la base para exportarla

In [11]:
df_final = df_concatenado.drop(columns=['cve_ent', 'codigo_act'])

carpeta_final = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"

df_final.to_csv(os.path.join(carpeta_final, "restaurantessolo_limpio.csv"), index=False, encoding='utf-8-sig')


Para obtener el archivo de "restaurantes_limpio" es necesario borrar la linea de código que acota los códigos de la columna "código_act" que se recuperaran de la base de datos. La linea tiene un comentario con una flecha: "######### <------ es esta"